In [1]:
from gensim.models.fasttext import FastText
import pandas as pd
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Train FastText

In [2]:
train_df1 = pd.read_csv("train_supervised_dataset.csv")
train_df1["source"] = "super"
train_df2 = pd.read_csv("train_unsupervised_dataset.csv")
train_df2["source"] = "unsuper"
test_df = pd.read_csv("test_dataset.csv")
test_df["source"] = "test"

In [3]:
# set(" ".join(train_df1[~train_df1.good.isna()].good.tolist())) # оставляем -

In [4]:
# set(" ".join(train_df1[~train_df1.brand.isna()].brand.tolist())) # оставляем все

In [5]:
names = pd.concat((train_df1, test_df, train_df2)).reset_index(drop=True).drop(columns=["id"])
names = names[names.name != "Тестирование 11"]
names = names[names.name != "Взнос на лицевой счет от: 11"]
names = names[names.name != "1  11"]
print(names.shape)
names = names.drop_duplicates()
print(names.shape)
names.head()

(1029988, 4)
(1029943, 4)


,name,good,brand,source
0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax,super
1,87191 Бусы для елки шарики_87191,бусы,NaN,super
2,Футболка Piazza Italia WR011446881,футболка,piazza italia,super
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN,super
4,Одежда (вес) 1500,одежда,NaN,super


In [6]:
# non_unique_names = {k: v for k, v in names.name.value_counts().to_dict().items() if v > 1}
# for k, v in non_unique_names.items():
#     print(f"|{k}|")
#     display(names[names.name == k])

In [7]:
names = names.drop_duplicates(subset=["name"])
print(names.shape)
names.head()

(1029849, 4)


,name,good,brand,source
0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax,super
1,87191 Бусы для елки шарики_87191,бусы,NaN,super
2,Футболка Piazza Italia WR011446881,футболка,piazza italia,super
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN,super
4,Одежда (вес) 1500,одежда,NaN,super


In [8]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub("_", " ", text)
    text = " " + text + " "
    text = re.sub("\d{5,}", " ", text) # убираю длиньше чем есть в метках
    text = re.sub("\d+[\.\,]?\d* *(см|гр|г|л|мл|кг|шт|мкм|м)", " ", text) # убираю единицы измерения
    text = re.sub("\(.*\)", " ", text) # убираем скобки и содержимое
    text = re.sub("\[.*\]", " ", text) # убираем скобки и содержимое
    text = re.sub("<.+>", " ", text) # убираем скобки и содержимое
    text = re.sub("{.+}", " ", text) # убираем скобки и содержимое
    text = re.sub("\d+[\.\,]?\d*\s?%", " ", text) # процентное содержание в товарах

    text = re.sub("\x01", "е", text)
    text = re.sub("\x07", " ", text)
    text = re.sub("\t", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub("\x18", " ", text)
    text = re.sub("\x1a", " ", text)
    text = re.sub('"', " ", text)
    text = re.sub("#", "", text)
    text = re.sub("\$", "", text)
    text = re.sub('%', "", text)
    text = re.sub('\(', "", text)
    text = re.sub('\)', "", text)
    text = re.sub('\*', "", text)
    text = re.sub(",", " ", text)
    text = re.sub("/", " ", text)
    text = re.sub(":", " ", text)
    text = re.sub(";", " ", text)
    text = re.sub("<", "", text)
    text = re.sub("=", "", text)
    text = re.sub(">", "", text)
    text = re.sub('\?', "", text)
    text = re.sub("@", "", text)
    text = re.sub('\[', "", text)
    text = re.sub('\\\\', " ", text)
    text = re.sub('\]', "", text)
    text = re.sub('\^', "", text)
    text = re.sub('{', " ", text)
    text = re.sub('}', " ", text)
    text = re.sub('\|', " ", text)
    text = re.sub('~', " ", text)
    text = re.sub("\x7f", " ", text)
    text = re.sub("\xa0", " ", text)
    text = re.sub("¤", "", text)
    text = re.sub("°", " ", text)
    text = re.sub("·", " ", text)
    text = re.sub("є", " ", text)
    text = re.sub("ї", "", text)
    text = re.sub("ў", "", text)
    text = re.sub("∙", " ", text)
    text = re.sub("┬", "", text)
    text = re.sub("╗", "", text)
    text = re.sub("╣", " ", text)
    text = re.sub("░", "", text)

    text = re.sub(" +", " ", text)
    return text.strip()

In [9]:
names.name = names.name.parallel_apply(preprocess_text)
names.head()

,name,good,brand,source
0,petmax бантик леопард с красн розой,бантик,petmax,super
1,бусы для елки шарики,бусы,NaN,super
2,футболка piazza italia wr,футболка,piazza italia,super
3,7 yi572-03x-one заколка для волос для девочки,заколка,NaN,super
4,одежда 1500,одежда,NaN,super


In [10]:
# set(" ".join(names[~names.name.isna()].name.tolist()))

# "\x01" "е"
# "\x07" " "
# "\t" " "
# "\n" " "
# "\x18" " "
# "\x1a" " " странный символ
# '"' " " подумать что с этим делать
# "#" ""
# "\$" ""
# '%' ""
# '\(' ''
# '\)' ''
# '\*' ''
# "," " "
# "/" " "
# ":" " "
# ";" " "
# "<" ""
# "=" "" указывает всякие длины и диаметры можно подумать над регуляркой
# ">" ""
# '\?' ""
# "@" ""
# '\[' ""
# '\\\\' " " тут есть сокращения можно подумать над регуляркой
# '\]' ""
# '\^' "" странный символ посмотреть более подробно
# '{' " "
# '}' " "
# '\|' " "
# '~' " "
# "\x7f" " "
# "\xa0" " "
# "¤" ""
# "°" " " знак градусов можно подумать над регуляркой
# "·" " "
# "є" " "
# "ї" ""
# "ў" ""
# "∙" " "
# "┬" "" посмотреть на наличие регулярок
# "╗" ""
# "╣" " "
# "░" ""

In [11]:
# char = '░'
# names[names.name.str.contains(char)]

In [12]:
# names.iloc[31758]["name"]

In [13]:
names["tokens"] = names["name"].str.split()
names

,name,good,brand,source,tokens
0,petmax бантик леопард с красн розой,бантик,petmax,super,"[petmax, бантик, леопард, с, красн, розой]"
1,бусы для елки шарики,бусы,NaN,super,"[бусы, для, елки, шарики]"
2,футболка piazza italia wr,футболка,piazza italia,super,"[футболка, piazza, italia, wr]"
3,7 yi572-03x-one заколка для волос для девочки,заколка,NaN,super,"[7, yi572-03x-one, заколка, для, волос, для, д..."
4,одежда 1500,одежда,NaN,super,"[одежда, 1500]"
...,...,...,...,...,...
1029995,f-2296 спонж д макияжа фигурный,NaN,NaN,unsuper,"[f-2296, спонж, д, макияжа, фигурный]"
1029996,4 69.88 дифф arom 1601,NaN,NaN,unsuper,"[4, 69.88, дифф, arom, 1601]"
1029997,матрас надувной 540 tropical bird запл.д ремон...,NaN,NaN,unsuper,"[матрас, надувной, 540, tropical, bird, запл.д..."
1029998,пододеяльник стм страйп 3-сп размер 796х страй...,NaN,NaN,unsuper,"[пододеяльник, стм, страйп, 3-сп, размер, 796х..."


In [14]:
FastText(names["tokens"], vector_size=300).save("fasttext_300.model")